In [0]:
!pip install pymorphy2

In [0]:
from pymorphy2 import MorphAnalyzer
from lxml import html
import nltk
nltk.download('stopwords')
from nltk import sent_tokenize
from nltk.corpus import stopwords
from string import punctuation
import gensim
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
from collections import Counter, defaultdict
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
import pandas as pd

In [0]:
morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))


def normalize(text):
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]
    return ' '.join(words)


def tokenize(text):
    words = [word.strip(punct) for word in text.lower().split()]
    return ' '.join(words)

  

Часть 1

Буду обучать w2v модель на основе собственноручно собранного корпуса статей РБК за 2019 г. (там около 51000 статей)

In [2]:
from google.colab import files
uploaded = files.upload()

Saving RBC_fin.zip to RBC_fin.zip


In [3]:
import zipfile
zipcorpus = zipfile.ZipFile('RBC_fin.zip')
zipcorpus.extractall()
zipcorpus.close

<bound method ZipFile.close of <zipfile.ZipFile filename='RBC_fin.zip' mode='r'>>

In [0]:
data = open('RBC_fin.txt').read().splitlines()

data_norm = [normalize(text) for text in data]

In [0]:
data_norm = [text for text in data_norm if text]

In [15]:
data_norm[:5]

['авария воронежский область погибнуть управлять директор оэмк дтп воронежский область погибнуть управлять директор оскольский элекрометаллургический комбинат оэмк александр тищенко водитель предприятие василий юмашев',
 'говориться сообщение холдинг металлоинвест рбк входить оэмк',
 'директор металлоинвест андрей варичев назвать смерть тищенко огромный утрата компания также выразить соболезнование семья погибший',
 'отмечаться тищенко 1980 год работать новолипецкий металлургический завод 2003-го стать заместитель главный инженер оэмк технический перевооружение',
 '2011 год главное инженер комбинат декабрь 2018 год стать управлять директор']

In [17]:
len(data_norm)

845659

In [0]:
w2v = gensim.models.Word2Vec([text.split() for text in data_norm], size=50, sg=1)

In [0]:
w2v.save("word2vec.model")

Скачаем предуобученную модель с Rusvectores: tayga_upos_skipgram_300_2_2019

In [114]:
from google.colab import files
uploaded = files.upload()

Saving 185.zip to 185.zip


In [0]:
import zipfile
zipcorpus = zipfile.ZipFile('185.zip')
zipcorpus.extractall()
zipcorpus.close

In [0]:
taiga = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

Скачаем данные соревнования по определению перефразирования с колонками нормализованных в mystem текстов с частереченой разметкой

In [150]:
from google.colab import files
uploaded = files.upload()

Saving data_paraphraser_norm.csv to data_paraphraser_norm.csv


In [0]:
data = pd.read_csv('data_paraphraser_norm.csv')

Нормализуем тексты также без частеречной разметки, т.к. в обученную на РБК модель подавались слова в начальной форме без тэгов

In [0]:
data['text_1_norm_no_pos'] = data['text_1'].apply(normalize)
data['text_2_norm_no_pos'] = data['text_2'].apply(normalize)

In [237]:
data.head(5)

,label,text_1,text_2,text_1_norm,text_2_norm,text_1_norm_no_pos,text_2_norm_no_pos
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский_NOUN разрешать_VERB стрелять_VERB ...,полиция_NOUN мочь_VERB разрешать_VERB стрелять...,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право_ADV полицейский_NOUN на_ADP проникновени...,правило_NOUN внесудебный_ADJ проникновение_NOU...,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент_NOUN египет_NOUN вводить_VERB чрезвы...,власть_NOUN египет_NOUN угрожать_VERB вводить_...,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,вернуться_VERB из_ADP сирия_NOUN россиянин_NOU...,самолет_NOUN мчс_NOUN вывозить_VERB россиянин_...,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,в_ADP москва_NOUN из_ADP сирия_NOUN вернуться_...,самолет_NOUN мчс_NOUN вывозить_VERB россиянин_...,москва сирия вернуться 2 самолёт мчс россиянин...,самолёт мчс вывезти россиянин разрушить сирия


Функция векторизации предложения/текста

In [0]:
def get_embedding(model, text, dim):
  text = text.strip(punct).split()
  words = Counter(text)
  total = len(text)
  vectors = np.zeros((len(words), dim))
  for i,word in enumerate(words):
    try:
      v = model[word]
      vectors[i] = v * (words[word]/total)
    except (KeyError, ValueError):
      #print('Что-то пошло не так')
      continue
    
  if vectors.any():
    vector = np.average(vectors, axis=0)
  else:
    vector = np.zeros((dim))
  return vector

In [0]:
get_embedding(w2v, 'слово слово текст работать функция', w2v.vector_size)

In [0]:
get_embedding(taiga, 'слово_NOUN слово_NOUN текст_NOUN работать_VERB функция_NOUN', taiga.vector_size)

Функция создания списка векторизованных текстов, на основе входящего списка нормализованных текстов и модели

In [0]:
def vectorized_column (column, model):
  dim = model.vector_size
  vectorized = []
  for text in column:
    vectorized += [get_embedding(model, text, dim)]
  return vectorized

Предскажем перефразирование на основе модели РБК

In [308]:
X_text_1 = vectorized_column (data['text_1_norm_no_pos'], w2v)
X_text_2 = vectorized_column (data['text_2_norm_no_pos'], w2v)
X_text = np.concatenate([X_text_1, X_text_2], axis = 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [0]:
y = data.label.values

In [0]:
clf = LogisticRegression(max_iter=200) 

In [311]:
cv_scores_RBC = cross_val_score(clf, X_text, y, scoring='f1_micro', cv=5)
print('cv scores:', cv_scores_RBC)
print('average cv_score:', np.mean(cv_scores_RBC))

cv scores: [0.45228216 0.48616874 0.5266436  0.40484429 0.40276817]
average cv_score: 0.4545413908790268


А теперь посмотрим как предсказывается перефразирование на основе модели корпуса Тайги

In [0]:
X_text_1 = vectorized_column (data['text_1_norm'], taiga)
X_text_2 = vectorized_column (data['text_2_norm'], taiga)
X_text = np.concatenate([X_text_1, X_text_2], axis = 1)

In [317]:
cv_scores_Taiga = cross_val_score(clf, X_text, y, scoring='f1_micro', cv=5)
print('cv scores:', cv_scores_Taiga)
print('average cv_score:', np.mean(cv_scores_Taiga))

cv scores: [0.46749654 0.50069156 0.50242215 0.40692042 0.39307958]
average cv_score: 0.4541220500892571


Результаты обеих моделей очень близки. Модель РБК оказалась даже немного лучше. Учитывая, что размер корпуса статей РБК за 2019 г.(51000 статей, если каждая из них по 100-200 слов, то это ~5-10 млн. слов) гораздо меньше корпуса Тайги(5 млрд. слов), результат скорее говорит в пользу первой модели. Парафразы в соревновании были взяты из новостных текстов, соответственно корпус новостных текстов лучше подходит для этой конкретной задачи. Могу предположить, что если была бы обучена  модель на основе корпуса художественной литературы (сопоставимого с РБК корпусом по размерам), то результат был бы хуже. Зная заранее тематику новостей в парафразах, можно было бы обучить модель на основе соответствующей рубрики (например, "Экономика" или "Латинская Америка") и улучшить результат.